# Project 5: mini machine learning project

Maaike de Jong 

Ironhack Amsterdam Data Analytics 2020

### Notebook 1: data wrangling


This project uses the data from project 2: Sustainability in Amsterdam

In this project I will use Machine Learning models to see to what extent green indicator variables can predict income and Amsterdam city district. My questions are:

Q1: How well do energy label scores and number of solar panels predict income?  
Q2: Can energy scores, solar panels and income predict the city district?

I used the following datasets:  
From the [maps data portal](https://maps.amsterdam.nl/open_geodata/) of the Amsterdam city council:

- Solar panels (Zonnepanelen)
- Postcodes (PC6_VLAKKEN_BAG.csv)
- Neighbourhoods (GEBIED_BUURTEN.csv)
- City districts (GEBIED_STADSDELEN.csv)

From [Overheid.nl](overheid.nl):

- Energylabels in Amsterdam
- Income per Amsterdam area
All datasets can be found in this [google folder](https://drive.google.com/drive/folders/19VhvQbT89SLKaLnWsP20jhrTrqCvwMbd) 

This is the first part of two notebooks, here I combine variables from different datasets into the dataset used for the analysis in notebook 2. 

In [57]:
# Import packages

import geopandas
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely import wkt

In [58]:
# Import energy label data
energy_labels = pd.read_csv('../data/Energielabels_selectie gemeentes Amsterdam 4-1-2012.csv', sep = ';')
energy_labels.head()

,Pand_postcode,Pand_huisnummer,Pand_huisnummer_toev,Pand_gebouwcode,PandVanMeting_registratiedatum,EPC_HBU_Id,EPC_CBU_Id,PandVanMeting_energieprestatieindex,Certificaatnummer,PandVanMeting_energieverbuik_m2,...,SBI_nummer,BIK_nummer,BIK_Beschrijving,PandvanOpname_postcode,PandvanOpname_huisnummer,PandvanOpname_huisnummer_toev,PandvanOpname_gebouwcode,woningtype,PrimaryLast,gebouwtype
0,1011BN,41,C,,3/24/2011,2165973,,"2,36",249134160,0,...,,,,1011BN,41,C,,Portiekwoning,Primary Case,woningbouw
1,1011ED,82,,15,9/29/2011,2433732,,"1,39",893770206,0,...,,,,1011ED,82,,15,Portiekwoning,Primary Case,woningbouw
2,1011ED,84,,16,9/29/2011,2433733,,"1,3",923079865,0,...,,,,1011ED,84,,16,Portiekwoning,Primary Case,woningbouw
3,1011ED,86,,17,9/29/2011,2433734,,"1,01",779373327,0,...,,,,1011ED,86,,17,Portiekwoning,Primary Case,woningbouw
4,1011ED,92,,18,9/29/2011,2433735,,"1,47",696698833,0,...,,,,1011ED,92,,18,Rijwoning tussen,Primary Case,woningbouw


In [59]:
energy_labels.shape

(151656, 27)

In [60]:
energy_labels['woningtype'].value_counts()

Portiekwoning                           97867
Galerijwoning                           30382
Rijwoning tussen                        14425
Twee-onder-een-kap / rijwoning hoek      4301
Maisonnette                              2371
Flatwoning (overig)                       924
                                          784
Vrijstaande woning                        443
Woongebouw met niet-zelfstandige woo      159
Name: woningtype, dtype: int64

In [61]:
# select postcode and energy class columns into new df
energy_labels_df = energy_labels[['Pand_postcode', 'PandVanMeting_energieklasse']]
energy_labels_df = energy_labels_df.rename(columns = {'PandVanMeting_energieklasse':'energy_class', 'Pand_postcode': 'postcode'})
energy_labels_df.head()

,postcode,energy_class
0,1011BN,E
1,1011ED,C
2,1011ED,B
3,1011ED,A
4,1011ED,C


In [62]:
# check energy_classes
labels_list = sorted(list(set(energy_labels_df['energy_class'])))
labels_list

['A', 'A+', 'A++', 'B', 'C', 'D', 'E', 'F', 'G']

In [63]:
# add extra column with energy classes converted to numerical score
energy_labels_df['energy_class_score'] = energy_labels_df['energy_class'].replace({'A++': 9, 'A+': 8, 'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1})
energy_labels_df.head()

,postcode,energy_class,energy_class_score
0,1011BN,E,3
1,1011ED,C,5
2,1011ED,B,6
3,1011ED,A,7
4,1011ED,C,5


In [64]:
# add buurtcodes
postcodes = pd.read_csv('../data/PC6_VLAKKEN_BAG.csv', sep = ';')
postcodes = postcodes.rename(columns = {'Postcode6':'postcode'})
postcodes.head()

,OBJECTNUMMER,postcode,Buurtcode,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 7
0,1,1047HD,B10g,"MULTIPOLYGON(((4.735727 52.407357,4.735698 52....","MULTIPOLYGON(((52.407357 4.735727,52.407409 4....",4.737866,52.406724,NaN
1,2,1047HB,B10g,"MULTIPOLYGON(((4.739624 52.419242,4.739596 52....","MULTIPOLYGON(((52.419242 4.739624,52.419294 4....",4.739692,52.417695,NaN
2,3,1047HG,B10g,"MULTIPOLYGON(((4.741631 52.405609,4.741602 52....","MULTIPOLYGON(((52.405609 4.741631,52.405662 4....",4.742172,52.405478,NaN
3,4,1047HK,B10g,"MULTIPOLYGON(((4.746484 52.39985,4.746461 52.3...","MULTIPOLYGON(((52.39985 4.746484,52.399883 4.7...",4.746901,52.400519,NaN
4,5,1047HH,B10g,"MULTIPOLYGON(((4.749176 52.410365,4.749014 52....","MULTIPOLYGON(((52.410365 4.749176,52.410378 4....",4.749334,52.410536,NaN


In [65]:
pc_select = postcodes[['postcode', 'Buurtcode']]

In [66]:
# join buurten 

energy_buurten = pd.merge(energy_labels_df, pc_select, on = 'postcode', how = 'left')
energy_buurten.head()

,postcode,energy_class,energy_class_score,Buurtcode
0,1011BN,E,3,A04b
1,1011ED,C,5,A04e
2,1011ED,B,6,A04e
3,1011ED,A,7,A04e
4,1011ED,C,5,A04e


In [67]:
# create df with average energy class scores by buurt

energy_buurt = energy_buurten.groupby('Buurtcode')[['energy_class_score']].mean().reset_index()

energy_buurt.head()

,Buurtcode,energy_class_score
0,A00a,4.222222
1,A00b,5.353846
2,A00c,3.531915
3,A00d,2.300000
4,A00e,4.956522


In [68]:
energy_buurt.shape

(418, 2)

In [69]:
#import solar panel csv as pandas dataframe
solar_panels = pd.read_csv('../data/ZONNEPANELEN2017.csv', sep = ';')
solar_panels.head()


,OBJECTNUMMER,Functie,Gedetecteerde_panelen,Vermogen,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 8
0,1,NietWonen,21.0,5000,"POINT(4.849801,52.380596)","POINT(52.380596,4.849801)",4.849801,52.380596,NaN
1,2,Wonen,9.4,2000,"POINT(4.980629,52.296628)","POINT(52.296628,4.980629)",4.980629,52.296628,NaN
2,3,Wonen,9.4,2000,"POINT(4.970498,52.2995)","POINT(52.2995,4.970498)",4.970498,52.299500,NaN
3,4,Wonen,9.4,2000,"POINT(4.941263,52.370706)","POINT(52.370706,4.941263)",4.941263,52.370706,NaN
4,5,NietWonen,35.0,9000,"POINT(4.831484,52.38012)","POINT(52.38012,4.831484)",4.831484,52.380120,NaN


In [70]:
solar_panels.shape

(4617, 9)

In [71]:
#Use shapely.wkt sub-module to parse wkt format
solar_panels['WKT_LAT_LNG'] = solar_panels['WKT_LAT_LNG'].str.replace(',',' ')

solar_panels['WKT_LAT_LNG'] = solar_panels['WKT_LAT_LNG'].apply(wkt.loads)

In [72]:
#convert to geodataframe
solar_gdf = geopandas.GeoDataFrame(solar_panels, geometry='WKT_LAT_LNG')

solar_gdf.head()

,OBJECTNUMMER,Functie,Gedetecteerde_panelen,Vermogen,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 8
0,1,NietWonen,21.0,5000,"POINT(4.849801,52.380596)",POINT (52.38060 4.84980),4.849801,52.380596,NaN
1,2,Wonen,9.4,2000,"POINT(4.980629,52.296628)",POINT (52.29663 4.98063),4.980629,52.296628,NaN
2,3,Wonen,9.4,2000,"POINT(4.970498,52.2995)",POINT (52.29950 4.97050),4.970498,52.299500,NaN
3,4,Wonen,9.4,2000,"POINT(4.941263,52.370706)",POINT (52.37071 4.94126),4.941263,52.370706,NaN
4,5,NietWonen,35.0,9000,"POINT(4.831484,52.38012)",POINT (52.38012 4.83148),4.831484,52.380120,NaN


In [73]:
#check whether the 'point' columns are the right datatypes
type(solar_gdf.WKT_LAT_LNG)

geopandas.geoseries.GeoSeries

In [74]:
# then do a spatial join with the buurten geodata
# Import file with buurten to area conversion
buurten = pd.read_csv('../data/GEBIED_BUURTEN.csv', sep = ';')
buurten.head()

,OBJECTNUMMER,Buurt_code,Buurt,Buurtcombinatie_code,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 10
0,1,F81d,Calandlaan/Lelylaan,F81,F,275360.0,"POLYGON((4.800801 52.355175,4.809055 52.356842...","POLYGON((52.355175 4.800801,52.356842 4.809055...",4.809697,52.355708,NaN
1,2,F81e,Osdorp Zuidoost,F81,F,519366.0,"POLYGON((4.818583 52.357519,4.818622 52.356295...","POLYGON((52.357519 4.818583,52.356295 4.818622...",4.811344,52.353736,NaN
2,3,F82a,Osdorp Midden Noord,F82,F,215541.0,"POLYGON((4.786657 52.362712,4.795326 52.364434...","POLYGON((52.362712 4.786657,52.364434 4.795326...",4.791792,52.362078,NaN
3,4,F82b,Osdorp Midden Zuid,F82,F,258379.0,"POLYGON((4.788293 52.359736,4.796917 52.36148,...","POLYGON((52.359736 4.788293,52.36148 4.796917,...",4.793781,52.358838,NaN
4,5,F82c,Zuidwestkwadrant Osdorp Noord,F82,F,240774.0,"POLYGON((4.790209 52.356207,4.799258 52.358027...","POLYGON((52.356207 4.790209,52.358027 4.799258...",4.795597,52.355523,NaN


In [75]:
#Use shapely.wkt sub-module to parse wkt format
#buurten['WKT_LAT_LNG'] = buurten['WKT_LAT_LNG'].str.replace(',',' ')

buurten['WKT_LAT_LNG'] = buurten['WKT_LAT_LNG'].apply(wkt.loads)

In [76]:
#convert to geodataframe
buurten_gdf = geopandas.GeoDataFrame(buurten, geometry='WKT_LAT_LNG')

In [77]:
#select relevant columns from solar_gdf
solar_select = solar_gdf[['Functie', 'Gedetecteerde_panelen', 'WKT_LAT_LNG']]
solar_select = solar_select.rename(columns = {'Gedetecteerde_panelen':'solar_panels'})
solar_select2 = solar_select[solar_select['Functie'] == 'Wonen']

In [78]:
solar_select2.head()

,Functie,solar_panels,WKT_LAT_LNG
1,Wonen,9.4,POINT (52.29663 4.98063)
2,Wonen,9.4,POINT (52.29950 4.97050)
3,Wonen,9.4,POINT (52.37071 4.94126)
7,Wonen,10.0,POINT (52.34602 4.79340)
8,Wonen,9.4,POINT (52.31611 4.96028)


In [79]:
#assign the WGS84 latitude-longitude coordinate system to the geoseries
solar_select2.crs = "EPSG:4326"

In [80]:
buurten_select = buurten_gdf[['Buurt_code', 'WKT_LAT_LNG']]
buurten_select.crs = "EPSG:4326"

In [81]:
#perform spatial join in geopandas
solar_buurten = geopandas.sjoin(buurten_select, solar_select2, how="left", op="contains")

In [82]:
solar_buurten.head()

,Buurt_code,WKT_LAT_LNG,index_right,Functie,solar_panels
0,F81d,"POLYGON ((52.35518 4.80080, 52.35684 4.80905, ...",299.0,Wonen,9.4
0,F81d,"POLYGON ((52.35518 4.80080, 52.35684 4.80905, ...",1099.0,Wonen,9.4
0,F81d,"POLYGON ((52.35518 4.80080, 52.35684 4.80905, ...",801.0,Wonen,9.4
0,F81d,"POLYGON ((52.35518 4.80080, 52.35684 4.80905, ...",1292.0,Wonen,9.4
0,F81d,"POLYGON ((52.35518 4.80080, 52.35684 4.80905, ...",2237.0,Wonen,9.4


In [83]:
# new df with number of solar panels per buurt 

solar_buurt = solar_buurten.groupby('Buurt_code')[['solar_panels']].sum().reset_index()
solar_buurt = solar_buurt.rename(columns = {'Buurt_code': 'Buurtcode'})
solar_buurt.head()

,Buurtcode,solar_panels
0,A00a,0.0
1,A00b,0.0
2,A00c,13.0
3,A00d,0.0
4,A00e,0.0


In [84]:
# join energy labels and green roof data

energy_solar_buurt = pd.merge(energy_buurt, solar_buurt, on = 'Buurtcode', how = 'inner')
energy_solar_buurt.head()

,Buurtcode,energy_class_score,solar_panels
0,A00a,4.222222,0.0
1,A00b,5.353846,0.0
2,A00c,3.531915,13.0
3,A00d,2.300000,0.0
4,A00e,4.956522,0.0


In [85]:
energy_solar_buurt.shape

(418, 3)

In [86]:
# add buurt stadsdeelcode, lat, long to this df

buurten_select = buurten[['Buurt_code', 'Stadsdeel_code','LNG', 'LAT']]
buurten_select = buurten_select.rename(columns = {'Buurt_code': 'Buurtcode'})
buurten_select.head()

,Buurtcode,Stadsdeel_code,LNG,LAT
0,F81d,F,4.809697,52.355708
1,F81e,F,4.811344,52.353736
2,F82a,F,4.791792,52.362078
3,F82b,F,4.793781,52.358838
4,F82c,F,4.795597,52.355523


In [87]:
# join this data to df

combined_data = pd.merge(energy_solar_buurt, buurten_select, on = 'Buurtcode', how = 'left')
combined_data.head()

,Buurtcode,energy_class_score,solar_panels,Stadsdeel_code,LNG,LAT
0,A00a,4.222222,0.0,A,4.900171,52.375723
1,A00b,5.353846,0.0,A,4.896214,52.373797
2,A00c,3.531915,13.0,A,4.898577,52.372857
3,A00d,2.300000,0.0,A,4.893950,52.370864
4,A00e,4.956522,0.0,A,4.895578,52.369559


In [88]:
# add stadsdeel namen
stadsdelen = pd.read_csv('../data/GEBIED_STADSDELEN.csv', sep = ';')
stadsdelen.head()

,OBJECTNUMMER,Stadsdeel_code,Stadsdeel,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 8
0,1,A,Centrum,8043500,"POLYGON((4.932973 52.3704,4.932942 52.370539,4...","POLYGON((52.3704 4.932973,52.370539 4.932942,5...",4.903712,52.373297,NaN
1,2,B,Westpoort,28991600,"POLYGON((4.885861 52.39937,4.882702 52.401695,...","POLYGON((52.39937 4.885861,52.401695 4.882702,...",4.807319,52.411465,NaN
2,3,E,West,10629900,"POLYGON((4.895084 52.388684,4.894675 52.389933...","POLYGON((52.388684 4.895084,52.389933 4.894675...",4.865216,52.377879,NaN
3,4,F,Nieuw-West,38015500,"POLYGON((4.850498 52.364232,4.850459 52.365189...","POLYGON((52.364232 4.850498,52.365189 4.850459...",4.802676,52.363591,NaN
4,5,K,Zuid,17274000,"POLYGON((4.914989 52.342139,4.914945 52.342421...","POLYGON((52.342139 4.914989,52.342421 4.914945...",4.866063,52.341721,NaN


In [89]:
stadsdelen_select = stadsdelen[['Stadsdeel_code', 'Stadsdeel']]

In [90]:
# join this data with main df into final df

final_data = pd.merge(combined_data, stadsdelen_select, on = 'Stadsdeel_code', how = 'left')
final_data.head()

,Buurtcode,energy_class_score,solar_panels,Stadsdeel_code,LNG,LAT,Stadsdeel
0,A00a,4.222222,0.0,A,4.900171,52.375723,Centrum
1,A00b,5.353846,0.0,A,4.896214,52.373797,Centrum
2,A00c,3.531915,13.0,A,4.898577,52.372857,Centrum
3,A00d,2.300000,0.0,A,4.893950,52.370864,Centrum
4,A00e,4.956522,0.0,A,4.895578,52.369559,Centrum


In [91]:
# save data file for future use:
final_data.to_csv('final_data.csv', index=False)

In [92]:
# now also add income data

# Import income data file
income = pd.read_excel('../data/2019_stadsdelen_3_15.xlsx', skiprows = [0,1,3,80,112,113])
income.head()

,wijk/std,inwoners (x 1.000),personen met inkomen in particuliere huishoudens (x 1.000),gemiddeld persoonlijk inkomen (x 1.000 euro),huishoudens (x 1.000),gemiddeld besteedbaar inkomen per huishouden (x 1.000 euro),index gestand. Inkomen huishoudens (Ned.=100)
0,A00 Burgwallen-Oude Zijde,4.2,3.6,35.2,2.6,34.8,102.9
1,A01 Burgwallen-Nieuwe Zijde,4.0,3.4,38.1,2.5,35.1,104.2
2,A02 Grachtengordel-West,6.4,5.1,72.7,3.6,67.8,193.6
3,A03 Grachtengordel-Zuid,5.4,4.4,69.8,3.1,74.1,206.9
4,A04 Nieuwmarkt/Lastage,9.6,8.0,41.3,5.7,40.3,114.0


In [93]:
income_df = income[['wijk/std', 'gemiddeld persoonlijk inkomen (x 1.000 euro)']]

In [94]:
income_df = income_df.rename(columns = {'gemiddeld persoonlijk inkomen (x 1.000 euro)':'mean_income (x 1.000 euro)'})
income_df.head()

,wijk/std,mean_income (x 1.000 euro)
0,A00 Burgwallen-Oude Zijde,35.2
1,A01 Burgwallen-Nieuwe Zijde,38.1
2,A02 Grachtengordel-West,72.7
3,A03 Grachtengordel-Zuid,69.8
4,A04 Nieuwmarkt/Lastage,41.3


In [95]:
income_df['area'] = income_df['wijk/std'].str.extract('([A-Z]\d\d)')
income_df['area_name'] = income_df['wijk/std'].str.replace('([A-Z]\d\d)', '')
income_df.head()

,wijk/std,mean_income (x 1.000 euro),area,area_name
0,A00 Burgwallen-Oude Zijde,35.2,A00,Burgwallen-Oude Zijde
1,A01 Burgwallen-Nieuwe Zijde,38.1,A01,Burgwallen-Nieuwe Zijde
2,A02 Grachtengordel-West,72.7,A02,Grachtengordel-West
3,A03 Grachtengordel-Zuid,69.8,A03,Grachtengordel-Zuid
4,A04 Nieuwmarkt/Lastage,41.3,A04,Nieuwmarkt/Lastage


In [96]:
income_df2 = income_df[['area', 'mean_income (x 1.000 euro)']]
income_df2.head()

,area,mean_income (x 1.000 euro)
0,A00,35.2
1,A01,38.1
2,A02,72.7
3,A03,69.8
4,A04,41.3


In [97]:
final_data.head()

,Buurtcode,energy_class_score,solar_panels,Stadsdeel_code,LNG,LAT,Stadsdeel
0,A00a,4.222222,0.0,A,4.900171,52.375723,Centrum
1,A00b,5.353846,0.0,A,4.896214,52.373797,Centrum
2,A00c,3.531915,13.0,A,4.898577,52.372857,Centrum
3,A00d,2.300000,0.0,A,4.893950,52.370864,Centrum
4,A00e,4.956522,0.0,A,4.895578,52.369559,Centrum


In [98]:
final_data2 = final_data.copy()

In [99]:
final_data2.head()

,Buurtcode,energy_class_score,solar_panels,Stadsdeel_code,LNG,LAT,Stadsdeel
0,A00a,4.222222,0.0,A,4.900171,52.375723,Centrum
1,A00b,5.353846,0.0,A,4.896214,52.373797,Centrum
2,A00c,3.531915,13.0,A,4.898577,52.372857,Centrum
3,A00d,2.300000,0.0,A,4.893950,52.370864,Centrum
4,A00e,4.956522,0.0,A,4.895578,52.369559,Centrum


In [100]:
final_data2['area'] = final_data2['Buurtcode'].str.extract('([A-Z]\d\d)')
final_data2.head()

,Buurtcode,energy_class_score,solar_panels,Stadsdeel_code,LNG,LAT,Stadsdeel,area
0,A00a,4.222222,0.0,A,4.900171,52.375723,Centrum,A00
1,A00b,5.353846,0.0,A,4.896214,52.373797,Centrum,A00
2,A00c,3.531915,13.0,A,4.898577,52.372857,Centrum,A00
3,A00d,2.300000,0.0,A,4.893950,52.370864,Centrum,A00
4,A00e,4.956522,0.0,A,4.895578,52.369559,Centrum,A00


In [101]:
final_data_income = pd.merge(final_data2, income_df2, on = 'area', how = 'left')
final_data_income.head()

,Buurtcode,energy_class_score,solar_panels,Stadsdeel_code,LNG,LAT,Stadsdeel,area,mean_income (x 1.000 euro)
0,A00a,4.222222,0.0,A,4.900171,52.375723,Centrum,A00,35.2
1,A00b,5.353846,0.0,A,4.896214,52.373797,Centrum,A00,35.2
2,A00c,3.531915,13.0,A,4.898577,52.372857,Centrum,A00,35.2
3,A00d,2.300000,0.0,A,4.893950,52.370864,Centrum,A00,35.2
4,A00e,4.956522,0.0,A,4.895578,52.369559,Centrum,A00,35.2


In [102]:
# Save final data file to use in analysis

final_data_income.to_csv('final_data_income.csv', index=False)